In [96]:
import pandas as pd
import glob
import regex as re
import querygen as qg

In [97]:
files = glob.glob("results/*.csv")
dfs = [pd.read_csv(f, index_col=0) for f in files]
files

['results/eval_results_DNB_zeroshot_NousResearch-Hermes-3-Llama-3.2-3B-GGUF.csv',
 'results/eval_results_DBpedia_zeroshot_NousResearch-Hermes-3-Llama-3.1-8B-GGUF.csv',
 'results/eval_results_UniProt_oneshot_NousResearch-Hermes-3-Llama-3.2-3B-GGUF.csv',
 'results/eval_results_DNB_zeroshot_NousResearch-Hermes-3-Llama-3.1-8B-GGUF.csv',
 'results/eval_results_DBpedia_oneshot_NousResearch-Hermes-3-Llama-3.1-8B-GGUF.csv',
 'results/eval_results_BTO_zeroshot_NousResearch-Hermes-3-Llama-3.2-3B-GGUF.csv',
 'results/eval_results_BTO_oneshot_NousResearch-Hermes-3-Llama-3.1-8B-GGUF.csv',
 'results/eval_results_UniProt_zeroshot_NousResearch-Hermes-3-Llama-3.1-8B-GGUF.csv',
 'results/eval_results_UniProt_oneshot_NousResearch-Hermes-3-Llama-3.1-8B-GGUF.csv',
 'results/eval_results_BTO_zeroshot_NousResearch-Hermes-3-Llama-3.1-8B-GGUF.csv',
 'results/eval_results_DBpedia_zeroshot_NousResearch-Hermes-3-Llama-3.2-3B-GGUF.csv',
 'results/eval_results_DNB_oneshot_NousResearch-Hermes-3-Llama-3.1-8B-GGUF.csv

In [98]:
dfs

[Empty DataFrame
 Columns: []
 Index: [],
 Empty DataFrame
 Columns: []
 Index: [],
 Empty DataFrame
 Columns: []
 Index: [],
 Empty DataFrame
 Columns: []
 Index: [],
    f1_score  precision    recall  ged  normed_ged  \
 0       0.5        1.0  0.333333  3.0         0.4   
 1       0.0        0.0  0.000000  5.0         0.0   
 2       0.4        0.5  0.333333  2.0         0.6   
 3       0.0        0.0  0.000000  3.0         0.0   
 4       0.0        0.0  0.000000  3.0         0.0   
 5       0.0        0.0  0.000000  3.0         0.0   
 6       0.4        0.5  0.333333  4.0         0.2   
 7       0.0        0.0  0.000000  5.0         0.0   
 8       0.4        0.5  0.333333  4.0         0.2   
 
                                        response  \
 0  A animal, born in nature, but died in a town   
 1  A animal, born in nature, but died in a town   
 2  A animal, born in nature, but died in a town   
 3      A step in the career of a musical artist   
 4      A step in the career o

In [99]:

def graph_from_erl(erl: qg.EnrichedEntitiesRelations):
    G = qg.nx.DiGraph()
    for node in erl.entities:
        G.add_node(node.identifier, label=node.type)
    for link in erl.relations:
        G.add_edge(
            link.entity,
            link.target,
            weight=link.link.instance_count,
            label=link.relation,
        )
    return G

In [100]:
for f, df in zip(files, dfs):
    # data=re.findall(r"results/eval_results_(\w+)_.*", f)[0]
    # prompt_type=re.findall(r"results/eval_results_(\w+)_.*", f)[0]
    model = re.findall(r"results/eval_results_\w+_(.+)\.csv", f)[0]
    # df['prompt_type'] = prompt_type
    df["model"] = (
        model.replace("_", " ")
        .replace("-", " ")
        .rstrip("-GGUF")
        .lstrip("NousResearch Hermes 3 Llama")
        .lstrip(".2")
        .lstrip(".3")
    )

df = pd.concat(dfs)
df

/tmp/ipykernel_3069580/2264217373.py:15: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  df = pd.concat(dfs)


,model,f1_score,precision,recall,ged,normed_ged,response,cfg_name,zeroshot,stage,erl,generator,n_nodes,seed,erl_loaded
0,1 8B,0.500000,1.000000,0.333333,3.0,0.400000,"A animal, born in nature, but died in a town",DBpedia,False,raw,"{""relations"":[{""entity"":""animal"",""relation"":""b...",human,3.0,60.0,"relations=[EnrichedRelation(entity='animal', r..."
1,1 8B,0.000000,0.000000,0.000000,5.0,0.000000,"A animal, born in nature, but died in a town",DBpedia,False,constrained,"{""relations"":[{""entity"":""animal"",""relation"":""b...",human,3.0,60.0,"relations=[EnrichedRelation(entity='animal', r..."
2,1 8B,0.400000,0.500000,0.333333,2.0,0.600000,"A animal, born in nature, but died in a town",DBpedia,False,aligned,"{""relations"":[{""entity"":""animal"",""relation"":""b...",human,3.0,60.0,"relations=[EnrichedRelation(entity='animal', r..."
3,1 8B,0.000000,0.000000,0.000000,3.0,0.000000,A step in the career of a musical artist,DBpedia,False,raw,"{""relations"":[{""entity"":""musical artist"",""rela...",human,3.0,12.0,relations=[EnrichedRelation(entity='musical ar...
4,1 8B,0.000000,0.000000,0.000000,3.0,0.000000,A step in the career of a musical artist,DBpedia,False,constrained,"{""relations"":[{""entity"":""musical artist"",""rela...",human,3.0,12.0,relations=[EnrichedRelation(entity='musical ar...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40,3B,0.000000,0.000000,0.000000,5.0,0.000000,"a work has a starring with actor, and a work h...",DBpedia,False,constrained,"{""relations"":[{""entity"":""work"",""relation"":""sta...",templated,3.0,2.0,"relations=[EnrichedRelation(entity='work', rel..."
41,3B,1.000000,1.000000,1.000000,0.0,1.000000,"a work has a starring with actor, and a work h...",DBpedia,False,aligned,"{""relations"":[{""entity"":""work"",""relation"":""sta...",templated,3.0,2.0,"relations=[EnrichedRelation(entity='work', rel..."
42,3B,0.333333,0.333333,0.333333,5.0,0.166667,"An Organisation member, who was once a part of...",DBpedia,False,raw,"{""relations"":[{""entity"":""Organisation member"",...",llama,3.0,3.0,relations=[EnrichedRelation(entity='Organisati...
43,3B,0.000000,0.000000,0.000000,6.0,-0.200000,"An Organisation member, who was once a part of...",DBpedia,False,constrained,"{""relations"":[{""entity"":""Organisation member"",...",llama,3.0,3.0,relations=[EnrichedRelation(entity='Organisati...


In [101]:
df

,model,f1_score,precision,recall,ged,normed_ged,response,cfg_name,zeroshot,stage,erl,generator,n_nodes,seed,erl_loaded
0,1 8B,0.500000,1.000000,0.333333,3.0,0.400000,"A animal, born in nature, but died in a town",DBpedia,False,raw,"{""relations"":[{""entity"":""animal"",""relation"":""b...",human,3.0,60.0,"relations=[EnrichedRelation(entity='animal', r..."
1,1 8B,0.000000,0.000000,0.000000,5.0,0.000000,"A animal, born in nature, but died in a town",DBpedia,False,constrained,"{""relations"":[{""entity"":""animal"",""relation"":""b...",human,3.0,60.0,"relations=[EnrichedRelation(entity='animal', r..."
2,1 8B,0.400000,0.500000,0.333333,2.0,0.600000,"A animal, born in nature, but died in a town",DBpedia,False,aligned,"{""relations"":[{""entity"":""animal"",""relation"":""b...",human,3.0,60.0,"relations=[EnrichedRelation(entity='animal', r..."
3,1 8B,0.000000,0.000000,0.000000,3.0,0.000000,A step in the career of a musical artist,DBpedia,False,raw,"{""relations"":[{""entity"":""musical artist"",""rela...",human,3.0,12.0,relations=[EnrichedRelation(entity='musical ar...
4,1 8B,0.000000,0.000000,0.000000,3.0,0.000000,A step in the career of a musical artist,DBpedia,False,constrained,"{""relations"":[{""entity"":""musical artist"",""rela...",human,3.0,12.0,relations=[EnrichedRelation(entity='musical ar...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40,3B,0.000000,0.000000,0.000000,5.0,0.000000,"a work has a starring with actor, and a work h...",DBpedia,False,constrained,"{""relations"":[{""entity"":""work"",""relation"":""sta...",templated,3.0,2.0,"relations=[EnrichedRelation(entity='work', rel..."
41,3B,1.000000,1.000000,1.000000,0.0,1.000000,"a work has a starring with actor, and a work h...",DBpedia,False,aligned,"{""relations"":[{""entity"":""work"",""relation"":""sta...",templated,3.0,2.0,"relations=[EnrichedRelation(entity='work', rel..."
42,3B,0.333333,0.333333,0.333333,5.0,0.166667,"An Organisation member, who was once a part of...",DBpedia,False,raw,"{""relations"":[{""entity"":""Organisation member"",...",llama,3.0,3.0,relations=[EnrichedRelation(entity='Organisati...
43,3B,0.000000,0.000000,0.000000,6.0,-0.200000,"An Organisation member, who was once a part of...",DBpedia,False,constrained,"{""relations"":[{""entity"":""Organisation member"",...",llama,3.0,3.0,relations=[EnrichedRelation(entity='Organisati...


In [102]:
df["prompt_type"] = df["zeroshot"].apply(lambda x: "zero shot" if x else "one shot")
df.drop(columns=["zeroshot"], inplace=True)
df

,model,f1_score,precision,recall,ged,normed_ged,response,cfg_name,stage,erl,generator,n_nodes,seed,erl_loaded,prompt_type
0,1 8B,0.500000,1.000000,0.333333,3.0,0.400000,"A animal, born in nature, but died in a town",DBpedia,raw,"{""relations"":[{""entity"":""animal"",""relation"":""b...",human,3.0,60.0,"relations=[EnrichedRelation(entity='animal', r...",one shot
1,1 8B,0.000000,0.000000,0.000000,5.0,0.000000,"A animal, born in nature, but died in a town",DBpedia,constrained,"{""relations"":[{""entity"":""animal"",""relation"":""b...",human,3.0,60.0,"relations=[EnrichedRelation(entity='animal', r...",one shot
2,1 8B,0.400000,0.500000,0.333333,2.0,0.600000,"A animal, born in nature, but died in a town",DBpedia,aligned,"{""relations"":[{""entity"":""animal"",""relation"":""b...",human,3.0,60.0,"relations=[EnrichedRelation(entity='animal', r...",one shot
3,1 8B,0.000000,0.000000,0.000000,3.0,0.000000,A step in the career of a musical artist,DBpedia,raw,"{""relations"":[{""entity"":""musical artist"",""rela...",human,3.0,12.0,relations=[EnrichedRelation(entity='musical ar...,one shot
4,1 8B,0.000000,0.000000,0.000000,3.0,0.000000,A step in the career of a musical artist,DBpedia,constrained,"{""relations"":[{""entity"":""musical artist"",""rela...",human,3.0,12.0,relations=[EnrichedRelation(entity='musical ar...,one shot
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40,3B,0.000000,0.000000,0.000000,5.0,0.000000,"a work has a starring with actor, and a work h...",DBpedia,constrained,"{""relations"":[{""entity"":""work"",""relation"":""sta...",templated,3.0,2.0,"relations=[EnrichedRelation(entity='work', rel...",one shot
41,3B,1.000000,1.000000,1.000000,0.0,1.000000,"a work has a starring with actor, and a work h...",DBpedia,aligned,"{""relations"":[{""entity"":""work"",""relation"":""sta...",templated,3.0,2.0,"relations=[EnrichedRelation(entity='work', rel...",one shot
42,3B,0.333333,0.333333,0.333333,5.0,0.166667,"An Organisation member, who was once a part of...",DBpedia,raw,"{""relations"":[{""entity"":""Organisation member"",...",llama,3.0,3.0,relations=[EnrichedRelation(entity='Organisati...,one shot
43,3B,0.000000,0.000000,0.000000,6.0,-0.200000,"An Organisation member, who was once a part of...",DBpedia,constrained,"{""relations"":[{""entity"":""Organisation member"",...",llama,3.0,3.0,relations=[EnrichedRelation(entity='Organisati...,one shot


In [103]:
df_clean = df.dropna()
df_clean.describe()

,f1_score,precision,recall,ged,normed_ged,n_nodes,seed
count,153.000000,153.000000,153.000000,153.000000,153.000000,153.000000,153.000000
mean,0.373747,0.388020,0.374292,5.424837,0.256487,3.529412,27.058824
std,0.400855,0.414491,0.408883,8.681266,0.357118,1.689981,48.324729
min,0.000000,0.000000,0.000000,0.000000,-0.346154,3.000000,0.000000
25%,0.000000,0.000000,0.000000,2.000000,0.000000,3.000000,3.000000
50%,0.333333,0.333333,0.333333,4.000000,0.142857,3.000000,9.000000
75%,0.700000,0.777778,0.700000,5.000000,0.400000,3.000000,15.000000
max,1.000000,1.000000,1.000000,68.000000,1.000000,10.000000,231.000000


In [104]:
df["erl_loaded"] = df["erl"].apply(qg.EnrichedEntitiesRelations.model_validate_json)
# df["response_loaded"] = df["response"].apply(lambda x: qg.EnrichedEntitiesRelations.model_validate_json(x) if isinstance(x, str) else None)
df["erl_graph"] = df["erl_loaded"].apply(graph_from_erl)
# df["response_graph"] = df["response"].apply(lambda x: graph_from_erl if x is not None else None)

In [105]:
# df["normed_ged"] = 1 - df["ged"] / df["erl_graph"].apply(
#     lambda x: (len(x.nodes) + len(x.nodes))
# )

In [106]:
means = df_clean.groupby(
    ["n_nodes", "prompt_type", "stage", "model", "generator", "cfg_name"]
)[["f1_score", "normed_ged"]].mean()
counts = df_clean.groupby(
    ["n_nodes", "stage", "prompt_type", "model", "generator", "cfg_name"]
)[["f1_score", "normed_ged"]].count()
means

f1_score  normed_ged
n_nodes prompt_type stage       model generator cfg_name                      
3.0     one shot    aligned      3B   human     DBpedia   0.633333    0.400000
                                      llama     BTO       0.609804    0.520098
                                                DBpedia   0.516667    0.435714
                                      templated BTO       0.633631    0.447794
                                                DBpedia   1.000000    0.916667
                                1 8B  human     DBpedia   0.266667    0.266667
                    constrained  3B   human     DBpedia   0.000000    0.000000
                                      llama     BTO       0.000000   -0.007353
                                                DBpedia   0.000000   -0.135714
                                      templated BTO       0.000000    0.000000
                                                DBpedia   0.000000    0.000000
                                1 8B  human     DBpedia   0.000000    0.000000
                    raw          3B   human     DBpedia   0.522222    0.244444
                                      llama     BTO       0.262745    0.141176
                                                DBpedia   0.500000    0.091667
                                      templated BTO       0.656250    0.704167
                                                DBpedia   0.833333    0.642857
                                1 8B  human     DBpedia   0.300000    0.200000
5.0     one shot    aligned      3B   human     DBpedia   0.687831    0.215488
                    constrained  3B   human     DBpedia   0.000000   -0.127946
                    raw          3B   human     DBpedia   0.896296    0.269360
10.0    one shot    aligned      3B   human     DBpedia   0.700926    0.241758
                    constrained  3B   human     DBpedia   0.000000   -0.210623
                    raw          3B   human     DBpedia   0.652219    0.170426

In [107]:
counts

f1_score  normed_ged
n_nodes stage       prompt_type model generator cfg_name                      
3.0     aligned     one shot     3B   human     DBpedia          3           3
                                      llama     BTO             17          17
                                                DBpedia          4           4
                                      templated BTO             16          16
                                                DBpedia          2           2
                                1 8B  human     DBpedia          3           3
        constrained one shot     3B   human     DBpedia          3           3
                                      llama     BTO             17          17
                                                DBpedia          4           4
                                      templated BTO             16          16
                                                DBpedia          2           2
                                1 8B  human     DBpedia          3           3
        raw         one shot     3B   human     DBpedia          3           3
                                      llama     BTO             17          17
                                                DBpedia          4           4
                                      templated BTO             16          16
                                                DBpedia          2           2
                                1 8B  human     DBpedia          3           3
5.0     aligned     one shot     3B   human     DBpedia          3           3
        constrained one shot     3B   human     DBpedia          3           3
        raw         one shot     3B   human     DBpedia          3           3
10.0    aligned     one shot     3B   human     DBpedia          3           3
        constrained one shot     3B   human     DBpedia          3           3
        raw         one shot     3B   human     DBpedia          3           3

In [108]:
column_map = {
    "f1_score": "$F_1$",
    "normed_ged": "$GED_{s}$",
    "stage": "Stage",
    "prompt_type": "Prompt Type",
    "model": "Version \\& Size",
    "n_nodes": "$k$",
    "generator": "Query Origin",
}

## Compare Query Origins on DBpedia with 8B model

In [109]:
query_gen_compare = df_clean[
    df_clean["model"].str.contains("8B")
    & df_clean["cfg_name"].str.contains("DBpedia")
    & df_clean["prompt_type"].str.contains("one shot")
]
grouped_query_gen_compare = query_gen_compare.groupby(
    ["n_nodes", "generator", "stage"]
)[["f1_score", "normed_ged"]].mean()
grouped_query_gen_compare = grouped_query_gen_compare.rename(
    columns=column_map
).rename_axis([column_map["n_nodes"], column_map["generator"], column_map["stage"]], axis=0)
with open("tables/compare_gen_query.tex", "w") as f:
    f.write(
        grouped_query_gen_compare.to_latex(
            float_format="%.2f",
            caption="Comparison of query generation methods for LLama 3.1 8B on DBpedia with one shot prompts.",
            label="tab:compare_gen_query",
        )
    )
grouped_query_gen_compare

$F_1$  $GED_{s}$
$k$ Query Origin Stage                           
3.0 human        aligned      0.266667   0.266667
                 constrained  0.000000   0.000000
                 raw          0.300000   0.200000

## Compare models across Datasets

In [110]:
datasets = [
    "DBpedia",
    # "UniProt",
    "BTO",
    # "DNB",
]
grouped_dfs = {}
for dataset in datasets:
    model_compare = df_clean[df_clean["cfg_name"].str.strip() == dataset]
    grouped_model_compare = model_compare.groupby(
        ["n_nodes", "stage", "prompt_type", "model"]
    )[["f1_score", "normed_ged"]].mean()
    grouped_model_compare = grouped_model_compare.rename(
        columns=column_map
    ).rename_axis(
        [
            column_map["n_nodes"],
            column_map["stage"],
            column_map["prompt_type"],
            column_map["model"],
        ],
        axis=0,
    )
    grouped_dfs[dataset] = grouped_model_compare
    with open(f"tables/compare_model_{dataset}.tex", "w") as f:
        f.write(
            grouped_model_compare.to_latex(
                float_format="%.2f",
                caption=f"Comparison of models for {dataset}",
                label=f"tab:compare_model_{dataset}",
                # sparsify=False
            )
        )
dataset_keys = list(grouped_dfs.keys())
merged_grouped: pd.DataFrame = grouped_dfs[dataset_keys[0]].copy()
merged_grouped.rename(
    columns={
        "$F_1$": f"$F_1$_{dataset_keys[0]}",
        "$GED_{s}$": f"$GED_{{s}}$_{dataset_keys[0]}",
    },
    inplace=True,
)
for dataset, grouped_df in list(grouped_dfs.items())[1:]:
    print(f"Merging {dataset} into the grouped DataFrame")
    merged_grouped = pd.merge(
        merged_grouped,
        grouped_dfs[dataset].rename(
            columns={
                "$F_1$": f"$F_1$_{dataset}",
                "$GED_{s}$": f"$GED_{{s}}$_{dataset}",
            }
        ),
        on=["$k$", "Stage", "Prompt Type", "Version \\& Size"],
        # suffixes=(None, f"_{dataset}"),
    )
rename_map = {f"$F_1$_{dataset}": f"{dataset} $F_1$" for dataset in datasets} | {
    f"$GED_{{s}}$_{dataset}": f"{dataset} $GED_{{s}}$" for dataset in datasets
}
merged_grouped.rename(
    columns=rename_map,
    inplace=True,
)
with open("tables/compare_model_merged.tex", "w") as f:
    f.write(
        merged_grouped.to_latex(
            float_format="%.2f",
            caption="Comparison of models for DBpedia and UniProt",
            label="tab:compare_model_merged",
        )
    )

Merging BTO into the grouped DataFrame


In [111]:
grouped_dfs

{'DBpedia':                                                  $F_1$  $GED_{s}$
 $k$  Stage       Prompt Type Version \& Size                     
 3.0  aligned     one shot     3B              0.662963   0.530688
                              1 8B             0.266667   0.266667
      constrained one shot     3B              0.000000  -0.060317
                              1 8B             0.000000   0.000000
      raw         one shot     3B              0.581481   0.265079
                              1 8B             0.300000   0.200000
 5.0  aligned     one shot     3B              0.687831   0.215488
      constrained one shot     3B              0.000000  -0.127946
      raw         one shot     3B              0.896296   0.269360
 10.0 aligned     one shot     3B              0.700926   0.241758
      constrained one shot     3B              0.000000  -0.210623
      raw         one shot     3B              0.652219   0.170426,
 'BTO':                                           

In [112]:
merged_grouped

DBpedia $F_1$  DBpedia $GED_{s}$  \
$k$ Stage       Prompt Type Version \& Size                                     
3.0 aligned     one shot    3B                    0.662963           0.530688   
    constrained one shot    3B                    0.000000          -0.060317   
    raw         one shot    3B                    0.581481           0.265079   

                                             BTO $F_1$  BTO $GED_{s}$  
$k$ Stage       Prompt Type Version \& Size                            
3.0 aligned     one shot    3B                0.621356       0.485042  
    constrained one shot    3B                0.000000      -0.003788  
    raw         one shot    3B                0.453535       0.414141

In [113]:
df_clean["cfg_name"].str.strip() == "DBpedia"

0     True
1     True
2     True
3     True
4     True
      ... 
40    True
41    True
42    True
43    True
44    True
Name: cfg_name, Length: 153, dtype: bool